In [1]:
# Imports
import itertools
import functools
import multiprocess
import numpy as np

from research.data.data          import *
from research.data.utils         import *
from core.utils.entity_type      import Entity_type
from core.search.query_paper_mag import paper_mag_multiquery
from core.search.query_info      import paper_info_mag_check_multiquery

In [2]:
# Inputs
THREADS    = 8
BATCH_SIZE = 40

# List of authors
#author_name_list = ['stephen m blackburn', 'k r catchpole',
#                    'brendan d mckay', 'Andrew Blakers',
#                    'Brian D. O. Anderson', 'Wojciech Lipinski',
#                    'Robert C. Williamson', 'G.D. Farquhar',
#                    'Chennupati Jagadish', 'C. Jagadish',
#                    'Susan M. Scott', 'S. M. Scott',
#                    'Peter M.W. Gill', 'Philip Pettit',
#                    'Craig Moritz', 'David B. Lindenmayer',
#                    'David Lindenmayer', 'Anna Wierzbicka']
author_name_list = ['John W. Lloyd']
author_name_list = list(map(name_normalise, author_name_list))

# Additional paper ids to cache
extra_papers = []

In [3]:
# Get map from names -> author ids
name_maps = get_author_ids(author_name_list)

print('Number of author ids generated:', np.sum(list(map(len, name_maps.values()))))

return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be reple

In [4]:
# Generate paper ids to generate in cache
papers_to_cache = set()
for name, author_ids in name_maps.items():
    papers_to_cache.update(paper_mag_multiquery(Entity_type.AUTH, author_ids))
papers_to_cache.update(extra_papers)
papers_to_cache = list(papers_to_cache)

print('Number of paper ids generated:', len(papers_to_cache))

return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be reple

In [5]:
# Set up caching process for paper ids
p = multiprocess.Pool(THREADS)
    
# Set up for threads
batches = (papers_to_cache[i:i+BATCH_SIZE] for i in \
               range(0, len(papers_to_cache), BATCH_SIZE))

# Batch job with enumeration
def job(enum_input):
    batch_number, batch = enum_input
    print('Starting batch job:', batch_number, '| Size:', len(batch))
    res = paper_info_mag_check_multiquery(batch)
    print('Finishing batch job:', batch_number)
    return res

batch_res = p.map(job, list(enumerate(batches)))
paper_information = functools.reduce(lambda x, y: x + y, batch_res)

Starting batch job: 0 | Size: 40
Starting batch job: 1 | Size: 40
Starting batch job: 3 | Size: 40
Starting batch job: 4 | Size: 37
Starting batch job: 2 | Size: 40
To call API: 40
To call API: 40
To call API: 37
To call API: 39
To call API: 39
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
b'\n{ "error": { "code": "Quot

return statue: 429
return statue: 429
{'Ocp-Apim-Subscription-Key': '9c0c0c5e7758484b9ed8092034eac796'}
ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
{'Ocp-Apim-Subscription-Key': '9c0c0c5e7758484b9ed8092034eac796'}
return statue: 403
ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t    

ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
return statue: 403
ERROR: problem with the request.
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscriptio

ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-

ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 429
{'Ocp-Apim-Subscription-Key': '9c0c0c5e7758484b9ed8092034eac796'}
ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e9

...

return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 429
{'Ocp-Apim-Subscription-Key': '9c0c0c5e7758484b9ed8092034eac796'}
ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
return statue: 429
{'Ocp-Apim-Subscription-Key': 'aeea5a7bb8fd4d62af6c9c08b26521dc'}
ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t    

{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\

return statue: 429
{'Ocp-Apim-Subscription-Key': '9c0c0c5e7758484b9ed8092034eac796'}
ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later."

return statue: 403
{'Ocp-Apim-Subscription-Key': 'b76e92aa46114493a4a40ac52dd0c766'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 429
{'Ocp-Apim-Subscription-Key': '9c0c0c5e7758484b9ed8092034eac796'}
ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
return statue: 429
{'Ocp-Apim-Subscription-Key': '9c0c0c5e7758484b9ed8092034eac796'}
ERROR: problem with the request.
b'\n{ "error": { "code": "RateLimitExceeded", "message": "Rate limit is exceeded. Try again later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'adad58eaf8b94496a5d1b8c60a48fd48'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t    

Finishing batch job: 0
